In [ ]:
from difflib import HtmlDiff

df = HtmlDiff()

with open("./hoge.json") as f:
    f1 = f.readlines()

with open("./fuga.json") as f:
    f2 = f.readlines()
    
with open("./diff.html", "w") as f:
    f.writelines(df.make_file(f1, f2))

    


In [37]:
import diff_match_patch

class SideBySideDiff(diff_match_patch.diff_match_patch):

    def old_content(self, diffs):
        """
        Returns HTML representation of 'deletions'
        """
        html = []
        for (flag, data) in diffs:
            text = (data.replace("&", "&amp;")
                    .replace("<", "&lt;")
                    .replace(">", "&gt;")
                    .replace("\n", "<br>"))

            if flag == self.DIFF_DELETE:
                html.append("""<del style=\"background:#ffe6e6;
                    \">%s</del>""" % text)
            elif flag == self.DIFF_EQUAL:
                html.append("<span>%s</span>" % text)
        return "".join(html)

    def new_content(self, diffs):
        """
        Returns HTML representation of 'insertions'
        """
        html = []
        for (flag, data) in diffs:
            text = (data.replace("&", "&amp;")
                    .replace("<", "&lt;")
                    .replace(">", "&gt;")
                    .replace("\n", "<br>"))
            if flag == self.DIFF_INSERT:
                html.append("""<ins style=\"background:#e6ffe6;
                    \">%s</ins>""" % text)
            elif flag == self.DIFF_EQUAL:
                html.append("<span>%s</span>" % text)
        return "".join(html)

with open("./hoge.json") as f:
    old_string = f.read()

with open("./fuga.json") as f:
    new_string = f.read()

    
diff_obj = SideBySideDiff()
result = diff_obj.diff_main(old_string, new_string)
diff_obj.diff_cleanupSemantic(result)

distance = diff_obj.diff_levenshtein(result)

old_record = diff_obj.old_content(result) 
new_record = diff_obj.new_content(result)

with open("./old_diff.html", "w") as f:
    f.writelines(old_record)
with open("./new_diff.html", "w") as f:    
    f.writelines(new_record)

In [81]:
import glob
import yaml
import requests
import os

TEST_CASE_DIR = './test_case'
DEST_DIR = './report'

CURRENT_ASG = "http://yahoo.co.jp/"#os.environ['CURRENT_ASG']
NEW_ASG = "http://yahoo.co.jp/"#os.environ['NEW_ASG']
# ID = os.environ['ID']

# version 3.5 or more
test_cases = glob.glob(TEST_CASE_DIR + '/**/*.yml', recursive=True)

def request_data(data, url, request_type):
        title = data.get("title")
        current_url = url + data.get("path")
        r = requests.request(data.get("method").lower(), 
                             current_url, 
                             headers=data.get("headers"),
                             params=data.get("params"), 
                             data=data.get("body"))
        response = "StatusCode: " + str(r.status_code) + "\n"
        response += "\n"
        response += r.text
        
        directory = DEST_DIR + '/' + title
        
        if not os.path.exists(directory): os.mkdir(directory) 
        
        with open(directory + '/' + request_type + '.txt' , "w+") as f2:
            f2.writelines(response)


for fname in test_cases:
    with open(fname, "r+") as f:
        data = yaml.load(f)
        request_data(data, CURRENT_ASG, 'current') 
        request_data(data, NEW_ASG, 'new')
            
            
        
